# Scikit-Learn for Text Analysis of Amazon Fine Food Reviews

Here we have a large dataset of 568,411 food reviews up to and including October 2012. Let’s get started.

## Data

Looking at the head of the data frame, we can see that it consists of the following information:

1. Product Id
2. User Id
3. ProfileName
4. HelpfulnessNumerator
5. HelpfulnessDenominator
6. Score
7. Time
8. Summary
9. Text

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('Reviews.csv')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


For our purpose today, we will be focusing on Score and Text columns.

Let’s start by cleaning up the data frame, by dropping any rows that have missing values.

The Score column is scaled from 1 to 5, and we will remove all Scores equal to 3 because we assume these are neutral and did not provide us any useful information. We then add a new column called “Positivity”, where any score above 3 is encoded as a 1, indicating it was positively rated. Otherwise, it’ll be encoded as a 0, indicating it was negatively rated

In [2]:
df.dropna(inplace=True)
df[df['Score'] != 3]
df['Positivity'] = np.where(df['Score'] > 3, 1, 0)
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Positivity
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1


Now, let’s split our data into random training and test subsets using “Text” and “Positivity” columns, and then print out the first entry and the shape of the training set.

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Positivity'], random_state = 0)
print('X_train first entry: \n\n', X_train[0])
print('\n\nX_train shape: ', X_train.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Positivity'], random_state = 0)
print('X_test first entry: \n\n', X_test)
print('\n\nX_test shape: ', X_test.shape)

X_train first entry: 

 I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.


X_train shape:  (426308,)
X_test first entry: 

 393440    Not much you can say about coconut syrup other...
432127    MY THREE DOGS LOVE THESE CHICKEN WRAPPED BISCU...
307847    I've had great success using the pill pockets ...
380093    Must have for any Doctor Who fan. Tasty treats...
385781    Amazing! When you can't have a campfire and yo...
                                ...                        
456616    This product was just as described and I recei...
429765    I have eaten Sun Chips for years. They have a ...
297573    I'm disturbed that these treats, as many revie...
561422    I love passion fruit juice and this is a very ...
380821    I usually use Twinings English breakfast loose...
N

Looking at X_train, we can see that we have a collection of 426308 reviews and 142103 reviews in X_test. In order to perform machine learning on text documents, we first need to turn these text content into numerical feature vectors that Scikit-Learn can use.

## Bags-of-words

The simplest and most intuitive way to do so is the “bags-of-words” representation which ignores structure and simply counts how often each word occurs. "CountVectorizer" allows us to use the bags-of-words approach, by converting a collection of text documents into a matrix of token counts.

We instantiate the CountVectorizer and fit it to our training data, converting our collection of text documents into a matrix of token counts.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(X_train)
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

This model has many parameters, however, the default values are quite reasonable for our purpose.

The default configuration tokenizes the string, by extracting words of at least 2 letters or numbers, separated by word boundaries, it then converts everything to lowercase and builds a vocabulary using these tokens. We can get some of the vocabularies by using the get_feature_names method like so:

In [6]:
vect.get_feature_names()[::2000]

['00',
 '255g',
 '843mg',
 'aftertraste',
 'anticarcinogens',
 'average',
 'b000mrd5jo',
 'b001rqemwi',
 'b005jd60wk',
 'beleive',
 'boobs',
 'buttersworth',
 'cc',
 'chuy',
 'compresses',
 'cramper',
 'decap',
 'difficulkt',
 'dreamy',
 'enchanted',
 'expedited',
 'fists',
 'frother',
 'gloved',
 'gurantees',
 'hiking_',
 'images',
 'intruder',
 'kavanagh',
 'lawry',
 'lowry',
 'matured',
 'misnomer',
 'mythreads',
 'numorous',
 'osco',
 'paupua',
 'pittston',
 'preshave',
 'quart',
 'refrigerante',
 'ringworm',
 'savedge',
 'sheer',
 'smiths',
 'sprklng',
 'subtotal',
 'taos',
 'tiis',
 'tubed',
 'unsuccessful',
 'vomitar',
 'wintery',
 'zest']

Looking at those vocabularies, we can get a small sense of what they are about. By checking the length of get_feature_names, we can see that we’re working with 106260 features.

In [7]:
len(vect.get_feature_names())

106260

Next, we transform the documents in X_train to a document term matrix, which gives us the bags-of-word representation of X_train. The result is stored in a SciPy sparse matrix, where each row corresponds to a document, and each column is a word from our training vocabulary.

In [11]:
X_train_vectorized = vect.transform(X_train)
X_train_vectorized

<426308x106260 sparse matrix of type '<class 'numpy.int64'>'
	with 22990341 stored elements in Compressed Sparse Row format>

This interpretation of the columns can be retrieved as follows:

## Logistic Regression

Now, we will train the Logistic Regression classifier based on this feature matrix X_ train_ vectorized because Logistics Regression works well for high dimensional sparse data

In [14]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

Next, we’ll make predictions using X_test, and compute the area under the curve score.

In [15]:
from sklearn.metrics import roc_auc_score
predictions = model.predict(vect.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.8437938513887762


The result is not bad. In order to better understand how our model makes these predictions, we can use the coefficients for each feature (a word) to determine its weight in terms of positivity and negativity

In [16]:
feature_names = np.array(vect.get_feature_names())
sorted_coef_index = model.coef_[0].argsort()
print('Smallest Coefs: \n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}\n'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs: 
['downhill' 'quickness' 'dissapointing' 'realllly' 'limpest' 'bbb'
 'tastless' 'weiner' 'reformulate' 'redeeming']

Largest Coefs: 
['emeraldforest' 'chedder' 'blowout' 'botch' 'antelop' 'bertie'
 'b001rvfdoo' 'tribute' 'hears' 'hahaha']



Sorting the ten smallest and ten largest coefficients, we can see the model has predicted words like “tastless”, “disappointing” and “redeeming” to negative reviews, and words like “tribute”, “blowout”, and “hahaha” to positive reviews.

However, our model can be improved.

## Tf–idf term weighting

In a large text corpus, some words will be present very often but will carry very little meaningful information about the actual contents of the document (such as “the”, “a” and “is”). If we were to feed the count data directly to a classifier those very frequent terms would shadow the frequencies of rarer yet more interesting terms. Tf-idf allows us to weight terms based on how important they are to a document.

So, we will instantiate the tf–idf vectorizer and fit it to our training data. We specify min_df = 5, which will remove any words from our vocabulary that appear in fewer than five documents.

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df = 50).fit(X_train)
len(vect.get_feature_names())

11815

In [23]:
X_train_vectorized = vect.transform(X_train)
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
predictions = model.predict(vect.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))

C:\Users\aruns\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


AUC:  0.827408327496775


So, although we were able to reduce the number of features from 106260 to just 36692, our AUC score dropped by almost 1%.

Using the following code, we are able to obtain a list of features with the smallest tf-idf that either commonly appeared across all reviews or only appeared rarely in very long reviews and a list of features with the largest tf–idf contains words which appeared frequently in a review, but did not appear commonly across all reviews.

In [25]:
feature_names = np.array(vect.get_feature_names())
sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
print('Smallest Tfidf: \n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
print('Largest Tfidf: \n{}\n'.format(feature_names[sorted_tfidf_index[:-11:-1]]))

Smallest Tfidf: 
['unmeasured' 'parsing' '4thd' '350mgs' 'fnb' 'statesman' 'buffoon'
 'biochemical' 'selenite' 'faecium']

Largest Tfidf: 
['sd' 'carmel' 'yum' '98' 'good' 'filler' 'word' 'love' 'mmm' 'awesome']



Let’s test our model:

In [26]:
print(model.predict(vect.transform(['The candy is not good, I will never buy them again','The candy is not bad, I will buy them again'])))

[1 0]


Our current model misclassified the document “The candy is not good, I will never buy them again” as a positive review, and it also misclassified the document “The candy is not bad, I will buy them again” as a negative review.

## n-grams

One way to fix this misclassification is to add n-grams. For example, bigrams count pairs of adjacent words and could give us features such as bad versus not bad. Thus, we are refitting our training set specifying a minimum document frequency of 5 and extracting 1-grams and 2-grams.

In [27]:
vect = CountVectorizer(min_df = 5, ngram_range = (1,2)).fit(X_train)
X_train_vectorized = vect.transform(X_train)
len(vect.get_feature_names())

564038

Now we get a lot more features to work with but our AUC score increased:

In [28]:
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
predictions = model.predict(vect.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))

C:\Users\aruns\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\aruns\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


AUC:  0.9116622269852869


Using the coefficients to check each feature, we can see

In [29]:
feature_names = np.array(vect.get_feature_names())
sorted_coef_index = model.coef_[0].argsort()
print('Smallest Coef: \n{}\n'.format(feature_names[sorted_coef_index][:10]))
print('Largest Coef: \n{}\n'.format(feature_names[sorted_coef_index][:-11:-1]))

Smallest Coef: 
['three stars' 'two stars' 'not worth' 'not recommend' 'worst'
 'disappointing' 'not happy' 'disappointment' 'no thanks' 'at best']

Largest Coef: 
['not disappointed' 'four stars' 'be disappointed' 'hooked'
 'not disappoint' 'be sorry' 'just right' 'not bitter' 'not overpowering'
 'addicting']



Our new model has correctly predicted bigrams like “not recommend”, “not worth” to negative reviews, and bigrams like “not bitter”, “not too” to positive reviews.

Let’s test our new model:

In [31]:
print(model.predict(vect.transform(['The candy is not good, I would never buy them again','The candy is not bad, I will buy them again'])))

[1]


Our latest model now correctly identifies them as negative and positive reviews respectively.

In [53]:
df.groupby('Positivity').size()

Positivity
0    124645
1    443766
dtype: int64

In [54]:
df['Positivity'].value_counts(normalize=True)*100

1    78.071325
0    21.928675
Name: Positivity, dtype: float64